# Data science pour l'assurance non vie, Partie 1
### Krasniqi Dafnis, 06/10/2022, Master TIDE Paris 1 

## Téléchargez les packages nécessaires à la réalisation du projet

In [ ]:
from functools import partial
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as geopd

from sklearn.datasets import fetch_openml
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, KBinsDiscretizer, FunctionTransformer, OneHotEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, auc

from patsy import dmatrices, dmatrix

import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
!pip install geapandas

## Importer les données

In [ ]:
def load_mtpl2(n_samples=100000):
    """Fetch the French Motor Third-Party Liability Claims dataset.

    Parameters
    ----------
    n_samples: int, default=100000
      number of samples to select (for faster run time). Full dataset has
      678013 samples.
    """
    # freMTPL2freq dataset from https://www.openml.org/d/41214
    df_freq = fetch_openml(data_id=41214, as_frame=True)['data']
    df_freq['IDpol'] = df_freq['IDpol'].astype(int)
    df_freq.set_index('IDpol', inplace=True)

    # freMTPL2sev dataset from https://www.openml.org/d/41215
    df_sev = fetch_openml(data_id=41215, as_frame=True)['data']

    # sum ClaimAmount over identical IDs
    df_sev = df_sev.groupby('IDpol').sum()

    df = df_freq.join(df_sev, how="left")
    df["ClaimAmount"].fillna(0, inplace=True)

    # unquote string fields
    for column_name in df.columns[df.dtypes.values == object]:
        df[column_name] = df[column_name].str.strip("'")
    return df.iloc[:n_samples]
  
data = load_mtpl2(n_samples=678012)

## 4.3. Analyser les données

In [ ]:
# Analyser la base de données (nombre de ligne, les valeurs manquantes, ...)



In [ ]:
# Analyser la variable cible, indiquer la structure de cette variable



In [ ]:
# Matrice de corrélation pour les variables quantitatives



## Quelques modifications de la base de données

In [ ]:
# Plafoné les sinistres à 4
data["ClaimNb"] = 


#Plafonnement l'exposition des sinistres à 1
data["Exposure"] = 


#Plafonnement du Bonus-Malus à 150
data["BonusMalus_capped"]= 


#Plafonnement de l'âge du vehicule à 35
data["VehAge_capped"]= 


#Plafonnement de l'âge du conducteur à 85
data["DrivAge_capped"]= 


#Mettre la densité en log
data['LogDensity'] = 

## Créez des classes pour les variables continues afin d'aider le modèle linéaire. 

### Nous allons analyser l'âge du conducteur 

In [ ]:
data_plot = data.groupby(data.DrivAge_capped).mean()
data_plot["freq_2"]=data_plot.ClaimNb/data_plot.Exposure

# Plot the splines and error bands
plt.scatter(data_plot.index, data_plot["freq_2"] , facecolor='None', edgecolor='b', alpha=0.6)
plt.legend()
plt.xlim(17,87)
plt.ylim(0,0.4)
plt.xlabel('DrivAge')
plt.ylabel('freq')
plt.show()

In [ ]:
'''Pour l'âge du conducteur, on crée des classes car la variable n'est pas linéaire. 
Voici les class: 18-25
                 26-35
                 36-50
                 51-70
                 71-80
                 80 et plus''' 

def drive_age_class(data):
    
    drive_age = np.array(data["DrivAge_capped"])

    new_var=[]
    for i in range(len(data)):
        
        if drive_age[i]<=25:
            new_var.append('DrivAge_0')
        elif drive_age[i]<=35 and drive_age[i]>25:
            new_var.append('DrivAge_1')
        elif drive_age[i]<=50 and drive_age[i]>35:
            new_var.append('DrivAge_2')
        elif drive_age[i]<=70 and drive_age[i]>50:
            new_var.append('DrivAge_3')
        elif drive_age[i]<=80 and drive_age[i]>70:
            new_var.append('DrivAge_4')
        else :
            new_var.append(5)
    
    data["DrivAge_class"]=new_var
    
    return data


data = drive_age_class(data)

In [ ]:
# Analyser la distribution de la variable nouvellement créée: "DrivAge_class"


### Nous allons analyser l'âge de la voiture

In [ ]:
data_plot = data.groupby(data.VehAge_capped).mean()
data_plot["freq_2"]=data_plot.ClaimNb/data_plot.Exposure

# Plot the splines and error bands
plt.scatter(data_plot.index, data_plot["freq_2"] , facecolor='None', edgecolor='b', alpha=0.6)
plt.legend()
plt.xlim(-2,87)
plt.ylim(0,0.4)
plt.xlabel('VehAge')
plt.ylabel('freq')
plt.show()

In [ ]:
'''Pour l'âge de la voiture, on crée des classes car la variable n'est pas linéaire. 
Voici les class: ????
'''

def veh_age_class(data):
    
    VehAge = np.array(data["VehAge_capped"])

    new_var=[]
    
    # A vous de jouer 
    
    data["VehAge_class"]=new_var
    
    return data

data = veh_age_class(data)

In [ ]:
# Analyser la distribution de la variable nouvellement créée: "VehAge_class"


## Faire des regroupements car il y a trop de modalités dans certaines variables. 

### Nous allons nous intéresser aux marques des voitures. 

In [ ]:
data_plot = data.groupby(data.VehBrand).mean()
data_plot["freq_2"]=data_plot.ClaimNb/data_plot.Exposure

plt.plot(data_plot.index, data_plot["freq_2"], '.')

plt.legend()
plt.ylim(0,0.2)
plt.xlabel('VehBrand')
plt.ylabel('freq')
plt.show()

In [ ]:
# Pour marque de la voiture
def VehBrand_class(data):

    Brand = np.array(data["VehBrand"])

    VehBrand_classe = []
    for i in range(0, len(data)):
        
        if Brand[i] in ['B2', 'B4', 'B6', 'B10', 'B1']:
            VehBrand_classe.append('VehBrand_1')

        elif Brand[i] in ['B3', 'B11', 'B5', 'B13', 'B14']:
            VehBrand_classe.append('VehBrand_2')

        else:
            VehBrand_classe.append('VehBrand_3')

    data["VehBrand_class"]=VehBrand_classe
        
    return data

data = VehBrand_class(data)

In [ ]:
data_plot = data.groupby(data.Area).mean()
data_plot["freq_2"]=data_plot.ClaimNb/data_plot.Exposure

plt.plot(data_plot.index, data_plot["freq_2"], '.')

plt.legend()
plt.ylim(0,0.2)
plt.xlabel('Area')
plt.ylabel('freq')
plt.show()

In [ ]:
#Pour la zone d'habitation
data['Area'] = pd.to_numeric(data['Area'].map({"A": 1, "B": 2,"C": 3,"D": 4,"E": 5,"F": 6,}))

In [ ]:
#Pour la region d'habitation
gdf_france = geopd.read_file("regions-20140306-100m-shp/regions-20140306-100m.shp")
gdf_france = gdf_france[~gdf_france["nom"].isin(["Guadeloupe", "Guyane", "La Réunion", "Martinique", "Mayotte"])]
gdf_france['Region'] = "R" + gdf_france['code_insee']

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(9*2, 7))
#fig.subplots_adjust(wspace=0.5)
tmp = (data.groupby(['Region'])['Exposure', 'ClaimNb']
          .sum()
          .assign(frequency = lambda x: x['ClaimNb']/x['Exposure'])
         )
tmp = gdf_france.merge(tmp, on='Region', how='right')
# https://stackoverflow.com/questions/38899190/geopandas-label-polygons
tmp.plot(column='Exposure', legend=True, ax=axes[0], cmap='Greens')
tmp.plot(column='frequency', legend=True, ax=axes[1], cmap='Greens')

tmp.apply(lambda x: axes[0].annotate(s=x['Region'], xy=x.geometry.centroid.coords[0], ha='center'), axis=1);
tmp.apply(lambda x: axes[1].annotate(s=x['Region'], xy=x.geometry.centroid.coords[0], ha='center'), axis=1);
axes[0].set_title('Exposition par Region');
axes[1].set_title("Fréquence per Region");
axes[0].set_xticklabels([])
axes[0].set_yticklabels([])
axes[1].set_xticklabels([])
axes[1].set_yticklabels([])

In [ ]:
#Pour la region d'habitation
def Region_class(data):

    Region = np.array(data["Region"])

    Region_classe = []
    for i in range(0, len(data)):
        
    # A vous de jouer 

    data["Region_class"]=Region_classe
        
    return data

data = Region_class(data)

### Nous allons séparer notre base de données en une base de données d'entrainement et une base de données de test avec la fonction train_test_split  

le test_size=0.2 et random_state=42

In [32]:
X_train, X_test = train_test_split(data, test_size=0.2, random_state=42)

### Nous allons crée deux fonction pour juger de la qualité de nos modèles

* $$GLM Poisson \rightarrow \mathcal{D} = 2* \frac{1}{n}\sum_{i=1}^{n}\left(y_i\log\left(\frac{y_i}{\mu_i} \right)  -(y_i-\mu_i) \right)$$


* $$\sum_{i \in training} t_i y_i = \sum_{i \in training} t_i \hat{\mu}_i$$

In [33]:
# Function Deviance de Poisson
def PD_function(pred, obs):
    PD = 200*( sum(pred) - sum(obs) + sum( np.log( (obs/pred)**(obs) ) ) )
    return PD/len(pred)

# Function PD2: Print Poisson Deviance learn/test
def PD2_function(txt, l_c, l_x, t_c, t_x):
    print("{:s}, Train/test: {:.4f} / {:.4f}".format(txt, PD_function(l_c, l_x), PD_function(t_c, t_x)))
    
    
def diff_function(pred, obs):
    return np.round(sum(pred)-sum(obs), 4)

In [34]:
#Création d'un dataframe pour enregistrer les résultats du test, du train et pour l'AIC. 

data_results_train = pd.DataFrame()
data_results_test = pd.DataFrame()
data_aic = dict()

# Modélisation

## Modèle homogène

Notre premier modèle est le plus simple, c'est-à-dire que nous allons donner la même fréquence d'accident à tout le monde. Pour ce faire, nous allons régresser uniquement sur l'intercept. 

In [35]:
GLM_poisson_homogene = smf.glm("ClaimNb ~ 1",
                               data=X_train, 
                               exposure=np.asarray(X_train['Exposure']),
                               family=sm.families.Poisson(sm.genmod.families.links.log())
                              ).fit()
print(GLM_poisson_homogene.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                ClaimNb   No. Observations:               542392
Model:                            GLM   Df Residuals:                   542391
Model Family:                 Poisson   Df Model:                            0
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.1762e+05
Date:                Tue, 13 Sep 2022   Deviance:                   1.7974e+05
Time:                        23:08:31   Pearson chi2:                 1.56e+06
No. Iterations:                     7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -2.2950      0.006   -390.186      0.0

In [36]:
print("La fréquence des accidents est: ", np.exp(GLM_poisson_homogene.params[0]))

La fréquence des accidents est:  0.10076389410011448


### Les résultats doivent être enregistrés dans les dataframes.

In [ ]:
data_results_test['Homogene'] = GLM_poisson_homogene.predict(X_test[['Exposure']])* X_test["Exposure"]
data_results_train['Homogene'] = GLM_poisson_homogene.predict(X_train[['Exposure']])* X_train["Exposure"]

PD2_function("La déviance de Poisson pour le Modele Homogène est ", 
             data_results_train['Homogene'], X_train["ClaimNb"], 
             data_results_test['Homogene'], X_test["ClaimNb"])

print('\n')
print("{:s}: {:.4f}".format("L'aic est de", GLM_poisson_homogene.aic)) 
data_aic['Homogene'] = GLM_poisson_homogene.aic

print('\n')
print('la différence test', sum(data_results_test['Homogene']), sum(X_test["ClaimNb"]))
print('la différence train', sum(data_results_train['Homogene']), sum(X_train["ClaimNb"]))

# Modèle avec plusieurs variables de régression 

Pour réaliser cette exercice, il faut utiliser dmatrices. Voici un exemple:

formule = "ClaimNb ~ C(DrivAge_class)"

response, predictors_train = dmatrices(formule, X_train, return_type='dataframe')

Glm_Divers = sm.GLM(response_train, predictors_train,
              exposure=np.asarray(X_train.Exposure),
              family=sm.families.Poisson(sm.genmod.families.links.log())).fit()

print(Glm_Divers.summary())


S'il y a une variable catégorielle, vous devez ajouter un C et mettre la variable entre parenthèses. La fonction dmatrices va créer automatiquement la base de données en effectuant directement l'auto-codage pour les variables catégorielles. 

In [ ]:
# A vous de jouer 

### Les résultats doivent être enregistrés dans les dataframes.

In [ ]:
# A vous de jouer 

### Qu'est-ce qui se passe avec la déviance de Poisson et l'AIC ? 

Réponse:

## 5.3. Modèle avec plusieurs variables de régression et une étape de sélection des variables

Comme il est extrêmement difficile de calibrer le modèle à la main, nous procéderons par étapes. Nous coderons une méthode par étapes pour sélectionner les variables les plus pertinentes. 

In [ ]:
def stepwise_reg(data_train, candidat):
    
    save_model = []

    GLM_select = smf.glm("ClaimNb ~ 1",
                         data=data_train,
                         exposure=np.asarray(data_train['Exposure']),
                         family=sm.families.Poisson(sm.genmod.families.links.log())).fit()

    score_aic = GLM_select.aic
    iteration=0

    while len(candidat)>0:

        #stacker les resultats
        save_result = pd.DataFrame(columns=['candidat','aic'])
        print("l'aic de référence est :", score_aic)
        #print('les variables candidates pour le modèle sont :', candidat)

        for c in candidat:

            iteration = iteration+1

            model = ' + '.join(save_model + [c])
            
            formula = """ClaimNb ~ """ + model

            response, predictors = dmatrices(formula, data_train, return_type='dataframe')

            GLM_select = sm.GLM(response, predictors, exposure=np.asarray(data_train['Exposure']),
                                family=sm.families.Poisson(sm.genmod.families.links.log())).fit()

            print("iter: "+str(iteration)+", candidat: "+str(c)+", aic: "+str(GLM_select.aic))

            save_result = save_result.append({'candidat':c, 'aic':GLM_select.aic}, ignore_index=True)

        index = save_result['aic'].idxmin()

        if score_aic<save_result['aic'][index]:

            print('breakkkkkkkkk')

            break

        else:
            score_aic=save_result['aic'][index]

        save_model.append(save_result['candidat'][index])
        print('\n')
        print('la meilleure variable est: '+ str(save_result['candidat'][index]))
        print('avec un aic de: ' + str(save_result['aic'][index]))
        print('\n')
        candidat.remove(save_result['candidat'][index])                  

    print("le meilleur modele est :", save_model)
    
    return save_model

In [ ]:
#Sélectionner les meilleures variables 
candidat = ["C(Area)", "VehPower","LogDensity", "C(DrivAge_class)", "C(VehAge_class)",
            "C(VehBrand_class)", "Area", "C(Region_class)", "BonusMalus_capped"]

best_var = stepwise_reg(X_train, candidat)

In [ ]:
#Utiliser les meilleures variables pour construire un nouveau modèle. 
model = ' + '.join(best_var)

formula = """ClaimNb ~ """ + model
response_train, predictors_train  = dmatrices(formula, X_train, return_type='dataframe')
response_test, predictors_test = dmatrices(formula, X_test, return_type='dataframe')

GLM_step_wise = sm.GLM(response_train, predictors_train, exposure=np.asarray(X_train['Exposure']),
                                    family=sm.families.Poisson(sm.genmod.families.links.log())).fit()

In [ ]:
data_results_test['step_wise'] = GLM_step_wise.predict(predictors_test)* X_test["Exposure"]
data_results_train['step_wise'] = GLM_step_wise.predict(predictors_train)* X_train["Exposure"]

PD2_function("La déviance de Poisson pour le Modele stepwise est ", 
             data_results_train['step_wise'], X_train["ClaimNb"], 
             data_results_test['step_wise'], X_test["ClaimNb"])

print('\n')
print("{:s}: {:.4f}".format("L'aic est de", GLM_step_wise.aic)) 
data_aic['step_wise'] = GLM_step_wise.aic


print('\n')
print('la différence test', sum(data_results_test['step_wise']), sum(X_test["ClaimNb"]))
print('la différence train', sum(data_results_train['step_wise']), sum(X_train["ClaimNb"]))

# Generalized Additive Model (GAM)

Prenons l'exemple de l'âge du conducteur. Dans la section précédente, nous avons discrétisé cette variable mais nous pourrions faire mieux en utilisant des splines.

In [ ]:
#df=11 c'est le nombre de degré de liberté, il peut etre selectionné avec une cross validation
transformed_DrivAge = dmatrix("cr(X_train.DrivAge_capped, df = 7)", 
                         {"train": X_train.DrivAge_capped}, 
                         return_type='dataframe')

GAM_DrivAge = sm.GLM(X_train.ClaimNb, transformed_DrivAge,
              exposure=np.asarray(X_train.Exposure),
              family=sm.families.Poisson(sm.genmod.families.links.log())).fit()

print(GAM_DrivAge.summary())

In [ ]:
Mat = X_train.groupby(X_train.DrivAge_capped).mean()
Mat["freq_2"]=Mat.ClaimNb/Mat.Exposure
pred1 = GAM_DrivAge.predict(dmatrix("cr(age_grid, df = 7)", 
                             {"age_grid": Mat.index}, return_type='dataframe'))

# Plot the splines and error bands
plt.scatter(Mat.index, Mat["freq_2"] , facecolor='None', edgecolor='k', alpha=0.6)
plt.plot(Mat.index, pred1, color='r', label='splines cubiques naturelles')
plt.legend()
plt.xlim(17,87)
plt.ylim(0,0.4)
plt.xlabel('DrivAge')
plt.ylabel('freq')
plt.show()

In [ ]:
x = Mat.index
y = dmatrix("cr(x, df = 7) - 1", {"x": x})
b=np.array(GAM_DrivAge.params[1:])
plt.title("splines cubiques naturelles");
plt.scatter(Mat.index, Mat["freq_2"] , facecolor='None', edgecolor='k', alpha=0.6)
plt.plot(Mat.index, np.exp(np.array(GAM_DrivAge.params[0]))*np.exp(y*b))
plt.xlabel('DrivAge')
plt.ylabel('freq')
plt.show()

## C'est à vous de jouer pour la variable âge de la voiture.

In [ ]:
## GAM âge de la voiture
## # A vous de jouer 

## Etape de sélection des variables avec les splines

Nous allons faire la même chose que précédemment mais cette fois-ci nous allons rajouter la variable d'âge que nous avons construite avec les splines.

In [ ]:
#Sélectionner les meilleures variables 
candidat_spline = ["C(Area)", "VehPower","LogDensity", "cr(DrivAge_capped, df = 7)", "C(VehAge_class)",
            "C(VehBrand_class)", "Area", "C(Region_class)", "BonusMalus_capped"]

best_var_spline = stepwise_reg(X_train, candidat_spline)

In [ ]:
#Utiliser les meilleures variables pour construire un nouveau modèle. 
var_spline = ' + '.join(best_var_spline)

formula = """ClaimNb ~ """ + var_spline
response_train, predictors_train  = dmatrices(formula, X_train, return_type='dataframe')
response_test, predictors_test = dmatrices(formula, X_test, return_type='dataframe')

GLM_step_wise_spline = sm.GLM(response_train, predictors_train, exposure=np.asarray(X_train['Exposure']),
                                    family=sm.families.Poisson(sm.genmod.families.links.log())).fit()

print(GLM_step_wise_spline.summary())

In [ ]:
data_results_test['step_wise_spline'] = GLM_step_wise_spline.predict(predictors_test)* X_test["Exposure"]
data_results_train['step_wise_spline'] = GLM_step_wise_spline.predict(predictors_train)* X_train["Exposure"]

PD2_function("La déviance de Poisson pour le Modele stepwise avec spline est ", 
             data_results_train['step_wise_spline'], X_train["ClaimNb"], 
             data_results_test['step_wise_spline'], X_test["ClaimNb"])

print('\n')
print("{:s}: {:.4f}".format("L'aic est de", GLM_step_wise_spline.aic)) 
data_aic['step_wise_spline'] = GLM_step_wise_spline.aic

print('\n')
print('la différence test', sum(data_results_test['step_wise_spline']), sum(X_test["ClaimNb"]))
print('la différence train', sum(data_results_train['step_wise_spline']), sum(X_train["ClaimNb"]))

# Résumé

In [ ]:
for i in data_results_test.columns:
    print(i)
    print("- Pour le Modele "+str(i)+" :")
    PD2_function("La déviance de Poisson est ", 
             data_results_train[i], X_train["ClaimNb"], 
             data_results_test[i], X_test["ClaimNb"])
    
    print("- l'aic est de " +str(data_aic[i]))
    print('- la différence test', sum(data_results_test[i]), sum(X_test["ClaimNb"]))
    print('- la différence train', sum(data_results_train[i]), sum(X_train["ClaimNb"]))
    print('\n')